# Tutorial 07-01 Extracting Data from an API

In our work at GeoNinjas PythonAnalytics in the past three chapters, we've been working with 311 case data from San Francisco's Open Data site.  Up to this point, we've been working with a data export as a starting point for our analysis.  Let's imagine our colleagues are still interested in 311 case count.  They'd like to see it summarized by neighborhood.  Now, though, they're interested seeing very current data.  They want to only see open 311 cases from the previous day.  Luckily we can pull data directly from San Francisco's Open Data website.

For this tutorial, we're going to begin the first stage of a data pipeline.  We're going to be working with the same data source that we used for our previous two chapters' tutorials, but this time we're going straight to the source.  We will pull data directly from San Francisco's Open Data API.

#### 1. Import Packages

Let's start by importing the python packages that we'll need to extract our data.  We should be pretty familiar with pandas at this point (but if you need a refresher, check out Chapter 6 - Data Manipulation).  There are a couple new packages we're working with here though.  Let's go through them one by one.
- **requests** - This is a package designed to interact with web APIs.  We're going to use it to retrieve data from an open data portal
- **json** - This package is super helpful for converting long JSON strings to Python data types like strings, numbers, dictionaries, and lists.
- **datetime** - This package is going to help us work with (surprise) dates and times.  We're going to use it help choose which data we extract from our API.

In [ ]:
import pandas
import requests
import json
import datetime

#### 2. Get a Date for Seven Days Ago

Now let's use our Python skills and a new package to get yesterday's date.  This is a really common pattern in Python automation workflows.  Rather than hard-code or input a date, we can use the **datetime** package to find out what the current date and time are when our process is being executed.  Then we can use the current date to find relative dates (in our case, seven days ago).

Let's start by finding today's date.

In [ ]:
now = datetime.datetime.now()
now

The resulting datetime object we get is comprised of all the components of the date and time that the object was created.  In order, it lists the year, month, day, hour, minute, second, and millisecond

Now let's use a **timedelta** object to some math and subtract seven days.  That will give us the datetime for this time but seven days ago.

In [ ]:
start_date = now - datetime.timedelta(days=7)
start_date

You might notice that in the resulting datetime object, the day has changed, but the hour, minute, second, and millisecond are still the same.  That might be fine for some use-cases, but for our use-case we'd like to get data for the entire previous day.  Let's set the time to midnight instead.

We'll start by creating a **time** object with the default time (which happens to be midnight).  Then we'll combine the date and time to get an object that represents midnight seven days ago.

In [ ]:
# Set the time to midnight
midnight = datetime.time()

# Combine the date and time
start_day_midnight = datetime.datetime.combine(start_date, midnight)

start_day_midnight

Note that now, the hour and minute are set to 0 and there are no seconds or microseconds

#### 3. Format Our Request

One of the wonderful things about working with an API is that we can control some of the aspects of our data as we're ingesting it.  Every API is different, so it's worth reviewing API documentation (which is readily available in this case at [this link](https://dev.socrata.com/foundry/data.sfgov.org/vw6y-z8j6).  The parameters we'll use for our query are documented at [this page](https://dev.socrata.com/docs/queries/).

Let's think about this in terms of the anatomy of a SQL statement.
```
SELECT
    service_request_id,
    requested_datetime,
    status_notes,
    lat,
    long,
    neighborhoods_sffind_boundaries,
    source,
    supervisor_district,
    media_url,
    point
FROM
    DATA_SOURCE
WHERE
    status_description = 'Open' AND requested_datetime > CURRENT_DAY - 1

```



If we start from the top there, the first thing we see is a list of fields.  According to the documentation, we can format that as a comma separated list and pass it along with the URL we're going to.

In [ ]:
field_list = [
    'service_request_id',
    'requested_datetime',
    'status_notes',
    'lat',
    'long',
    'neighborhoods_sffind_boundaries',
    'source',
    'supervisor_district',
    'media_url',
    'point'
]

fields = "$select=" + ",".join(field_list)
fields

TIP - the `.join` method is a method of a string object in Python.  When you pass a list into that method, it will use the string to concatenate all the items in that list and turn them into a string.

Moving down through the SQL query, the next component we see is the `FROM` clause.  That's pretty straightforward in our API.  It's just going to be the root URL of our dataset.

In [ ]:
url = "https://data.sfgov.org/resource/vw6y-z8j6.json"

The next thing we notice moving down the SQL statement is the `WHERE` clause.  The API documentation shows a where clause that we can format similarly.  The first portion, where we restrict the records returned to only records where `status_description = 'Open'` is pretty straightforward, but the latter portion involving the `CURRENT_DAY` isn't.  That's where our datetime work from Step 2 comes into play.

We can format our datetime object into a string format that this API can use.  In this case, the API can read datetimes formatted like *YYYY-MM-DD T H:M:S*.  We'll use a method called **strftime** to return a string in this format.

In [ ]:
formatted_start_day = start_day_midnight.strftime('%Y-%m-%dT%H:%M:%S.%f') 
formatted_start_day

Now let's put that together with the rest of our `WHERE` clause.

In [ ]:
where = f"$where=status_description='Open' and requested_datetime > '{formatted_start_day}'"
where

Now we can put the components of our request all together in one URL.

In [ ]:
full_url = url+"?"+fields+"&"+where
full_url

#### 4. Make a request to the API

Now that we've got our URL formatted, we can actually make our request to the API.  Note that if you were to take that URL represented by the `full_url` variable and put it into a browser, you'd get a response from the API.  We're going to use the **requests** package to do the same thing with Python.

In [ ]:
response = requests.get(full_url)
response

Note that we get a **response** object as a result showing its `status_code` property.  These codes tell us whether our request was successful.  Two of the most common responses include 200 (successful) and 404 (not found).  [This Link](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes) contains a fairly comprehensive list of codes.

As long as the response code is in the 200s, we're good to go.  In this case, we're expecting a response that containing JSON information.  We'll call the `text` property to retrieve that data.

In [ ]:
response_text = response.text

Let's take a look at our results.  Before printing the entire response, we should probably see what the type of the response is and how long it is.  Then we can take a look at the first 1000 characters of the data to get an idea of what it looks like.

In [ ]:
print(type(response_text))
print(len(response_text))
response_text[0:1000]

This looks like we would expect for JSON data.  We can see it starts with a square bracket and then a curly bracket.  This tells me it's likely an array of objects.  The problem is that all that information is organized as a single string.  We'll need to change that into Python data types like lists and dictionaries to effectively work with it further.

#### 5. Format the Results of the Request

This is where our **json** package comes into play.  The json package can parse strings (or byte streams) and load our data into useful Python data types.  In this case, since our input is a string, we'll call the **loads** method.

In [ ]:
results_list = json.loads(response_text)
results_list[0:2]

Now our data is formatted nicely into recognizable and accessible Python data structures.  The array of features is represented as a list.  Each feature in that array is represented by a dictionary.  Now we can index into the list and access key/value pairs for each field in a feature.

That's useful, but it'd be way easier to manipulate that data if it was in a table.  So let's create a DataFrame out of the data.

In [ ]:
df = pandas.DataFrame(results_list)
df.head()